In [11]:
#####BUILDING THE RESEARCH FROM HERE. HAVE DETAILS NEEDED FOR THE ORIGINAL WORK

using JuMP
using Mosek
using MosekTools
model= Model(Mosek.Optimizer)
# Let parameter 
m=15 # num customers
NumVar=5 # num hospitals
ObjCoefficient=[1 1 1 1 1]
xs = [0, 5, -5, 4, -5]
ys = [0, 0.6, 0.5, -5, -4.5]
#r=[0,0,0,0,0]
ms = [0, 6.34, 6.3125, 10.25, 11.3125]
coverage=[1 0 0 0 1;0 0 0 1 1;1 0 0 0 1;0 0 1 1 1;0 0 0 1 1; 0 1 0 0 1;0 0 1 1 0;0 0 0 1 0;1 0 0 0 0;1 1 0 0 0;0 1 0 0 0;0 0 1 0 0;0 0 1 0 0;1 0 0 0 0;0 0 0 1 0 ] # here is the coverage matrix for five hospitals and two customers 


#create your variables
@variable(model, w[i=1:NumVar]>=0)
@variable(model, d[i=1:NumVar]>=0)
@variable(model, x[i=1:NumVar], Bin)
@variable(model, X>=0)
@variable(model, y>=0)
###########################
@variable(model, p[i=1:NumVar]>=0)
@variable(model, q[i=1:NumVar]>=0)
@variable(model, z[i=1:NumVar]>=0)
#@variable(model, r[i=1:NumVar]>=0)
@variable(model, R>=0)

#  x = [0, 5, -5, 4, -5] # x cordinates of the circle
#  y = [0, 0.6, 0.5, -5, -4.5] # y cordinates of the circle
 
coverage1=[[1, 0, 0, 0, 1],[0, 0, 0, 1, 1],[1, 0, 0, 0, 1],[0, 0, 1, 1, 1],[0, 0, 0, 1, 1], [0, 1, 0, 0, 1],[0, 0, 1, 1, 0],[0, 0, 0, 1, 0],[1, 0, 0, 0, 0],[1, 1, 0, 0, 0],[0, 1, 0, 0, 0],[0, 0, 1, 0, 0],[0, 0, 1, 0, 0],[1, 0, 0, 0, 0],[0, 0, 0, 1, 0]]
#A=[[2, 3, 4], [3, 2, 5]]
a=[-1, -1, 1.2, 1.2, 1.2, 0.8, 2.5, 2.5, -3, -3, -1.5, 3, 0.9, -3.7, 3.7] #x cordinate of the points
b=[0.2, 0.8, -0.85, 1.2, 0.5, -1.5, -1.3, 1.4, 1.5, -1.3, -0.8, -2.7, -2.4, 0.5, 0.8] #y cordinate of the points
r=[]        # array to store max distances for the center of each circle to each point
#print("A =", A)
#print("A =", coverage1)



function distance(xs, ys, a, b, coverage1) 
    r = Float64[]
    for (i,xv) in enumerate(xs)
        R = Float64[]
        for (j, jv) in enumerate(a)
            if coverage1[j][i] == 1
                push!(R, ((xs[i]-a[j])^2 + (ys[i]-b[j])^2)^0.5)
            end
         end
         if length(R) > 0
             push!(r, maximum(R))
         end
    end
    return r
end
r=distance(xs, ys, a, b, coverage1)

#Define your constraints
# function SmallCircleRadius(xs,ys,coverage)
#     return radius
# end

# function SmallCircleRadius(m)
#     r=0;
#     radius=[];
#     for i=1:m
#         if (i<=5)
#         r=r+i;
#         radius=[radius,"r"]
#         end
#     end
#     return radius
# end

# r=SmallCircleRadius(m)

for i in 1:NumVar
    @constraint(model,  w[i]>=d[i] + 1000x[i] - 1000)
    @constraint(model, w[i]  <= 1000x[i])
    @constraint(model, w[i]  <= d[i])
    ###################
    @constraint(model, p[i]  == X-xs[i])  
    @constraint(model, q[i]  == y-ys[i])
    @constraint(model, z[i]  == R-r[i])
    @constraint(model, [z[i], p[i], q[i]] in SecondOrderCone())
    #@constraint(model, z[i]^2>=x[i]^2+y[i]^2)
end

@constraint(model, constr[i=1:m],   sum(coverage[i,j]*x[j] for j=1:NumVar)  >= 1)
for k in 1:NumVar
   @constraint(model, X^2 + y^2 - d[k]-xs[k]*X-ys[k]*y <= -ms[k])
end


#Specify Objective function
@objective(model, Min, sum(ObjCoefficient[i]*w[i] for i=1:NumVar))

@show model
print(model)
optimize!(model)
@show termination_status(model)
@show primal_status(model)
@show dual_status(model)
@show objective_value(model)
for i in 1:NumVar
   println("x[$i] = ", value(x[i]))
    println("w[$i] = ", value(w[i]))
    println("r[$i] = ", value(r[i]))
end
@show value(X)
@show value(y)
@show value(R)


model = A JuMP Model
Minimization problem with:
Variables: 33
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 15 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 20 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 10 constraints
`QuadExpr`-in-`MathOptInterface.LessThan{Float64}`: 5 constraints
`Vector{VariableRef}`-in-`MathOptInterface.SecondOrderCone`: 5 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 28 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 5 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Mosek
Names registered in the model: R, X, constr, d, p, q, w, x, y, z
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 65              
  Cones                  : 10              
  Scalar variables       : 53           

370.5204925840102

In [8]:
#####BEGINNING OF ORIGINAL WORK


using JuMP
using Mosek
using MosekTools
model= Model(Mosek.Optimizer)
# Let parameter 
m=15 # num customers
NumVar=5 # num hospitals
ObjCoefficient=[1 1 1 1 1]
xs = [0, 5, -5, 4, -5]
ys = [0, 0.6, 0.5, -5, -4.5]
ms = [0, 6.34, 6.3125, 10.25, 11.3125]
coverage=[1 0 0 0 1;0 0 0 1 1;1 0 0 0 1;0 0 1 1 1;0 0 0 1 1; 0 0 1 0 1;0 0 1 1 0;0 0 0 1 0;1 0 0 0 0;1 1 0 0 0;1 1 0 0 1;0 0 1 0 0;0 0 1 0 0;1 0 0 0 0;0 0 0 1 0 ] # here is the coverage matrix for five hospitals and two customers 
xc=[0 2.5 -2.5 2 -2.5]
yc=[0 0.3 0.25 -2.5 -2.25]
r=[0, 0, 0, 0, 0]
#create your variables
@variable(model, w[i=1:NumVar]>=0)
@variable(model, d[i=1:NumVar]>=0)
@variable(model, x[i=1:NumVar], Bin)
@variable(model, X>=0)
@variable(model, y>=0)

###########################
@variable(model, p[i=1:NumVar]>=0)
@variable(model, q[i=1:NumVar]>=0)
@variable(model, z[i=1:NumVar]>=0)
#@variable(model, r[i=1:NumVar]>=0)
@variable(model, R>=0)

#Define your constraints

for i in 1:NumVar
    @constraint(model,  w[i]>=d[i] + 1000x[i] - 1000)
    @constraint(model, w[i]  <= 1000x[i])
    @constraint(model, w[i]  <= d[i])
     ###################
    @constraint(model, p[i]  == X-xs[i])  
    @constraint(model, q[i]  == y-ys[i])
    @constraint(model, z[i]  == R-r[i])
    @constraint(model, [z[i], p[i], q[i]] in SecondOrderCone())
    
    
end

@constraint(model, constr[i=1:m],   sum(coverage[i,j]*x[j] for j=1:NumVar)  >= 1)
for k in 1:NumVar
 #  @constraint(model, X^2 + y^2 - d[k]-xs[k]*X-ys[k]*y <= -ms[k])
    @constraint(model, (X-xc[k])^2 + (y-yc[k])^2 <= d[k])
end


#Specify Objective function
@objective(model, Min, sum(ObjCoefficient[i]*w[i] for i=1:NumVar))

@show model
print(model)
optimize!(model)
@show termination_status(model)
@show primal_status(model)
@show dual_status(model)
@show objective_value(model)
for i in 1:NumVar
   println("x[$i] = ", value(x[i]))
    println("w[$i] = ", value(w[i]))
    println("r[$i] = ", value(r[i]))
end
@show value(X)
@show value(y)
@show value(R)

model = A JuMP Model
Minimization problem with:
Variables: 33
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 15 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 20 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 10 constraints
`QuadExpr`-in-`MathOptInterface.LessThan{Float64}`: 5 constraints
`Vector{VariableRef}`-in-`MathOptInterface.SecondOrderCone`: 5 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 28 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 5 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Mosek
Names registered in the model: R, X, constr, d, p, q, w, x, y, z
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 65              
  Cones                  : 10              
  Scalar variables       : 53           

373.91262010975726

In [9]:
###CONTINUATION OF RESEARCH WORK WHERE WE FINF THE MAXIMUM DISTANCE FROM THE CENTRE TO A POINT IN THE CIRCLE 

using JuMP
using Mosek
using MosekTools
model= Model(Mosek.Optimizer)
# Let parameter 
m=15 # num customers
NumVar=5 # num hospitals
ObjCoefficient=[1 1 1 1 1]
xs = [0, 5, -5, 4, -5]
ys = [0, 0.6, 0.5, -5, -4.5]
ms = [0, 6.34, 6.3125, 10.25, 11.3125]
coverage=[1 0 0 0 1;0 0 0 1 1;1 0 0 0 1;0 0 1 1 1;0 0 0 1 1; 0 0 1 0 1;0 0 1 1 0;0 0 0 1 0;1 0 0 0 0;1 1 0 0 0;1 1 0 0 1;0 0 1 0 0;0 0 1 0 0;1 0 0 0 0;0 0 0 1 0 ] # here is the coverage matrix for five hospitals and two customers 
xc=[0 2.5 -2.5 2 -2.5]
yc=[0 0.3 0.25 -2.5 -2.25]

#create your variables
@variable(model, w[i=1:NumVar]>=0)
@variable(model, d[i=1:NumVar]>=0)
@variable(model, x[i=1:NumVar], Bin)
@variable(model, X>=0)
@variable(model, y>=0)

###########################
@variable(model, p[i=1:NumVar]>=0)
@variable(model, q[i=1:NumVar]>=0)
@variable(model, z[i=1:NumVar]>=0)
#@variable(model, r[i=1:NumVar]>=0)
@variable(model, R>=0)

#Define your constraints

for i in 1:NumVar
    @constraint(model,  w[i]>=d[i] + 1000x[i] - 1000)
    @constraint(model, w[i]  <= 1000x[i])
    @constraint(model, w[i]  <= d[i])
     ###################
    @constraint(model, p[i]  == X-xs[i])  
    @constraint(model, q[i]  == y-ys[i])
    @constraint(model, z[i]  == R-r[i])
    @constraint(model, [z[i], p[i], q[i]] in SecondOrderCone())
    
    
end

@constraint(model, constr[i=1:m],   sum(coverage[i,j]*x[j] for j=1:NumVar)  >= 1)
for k in 1:NumVar
 #  @constraint(model, X^2 + y^2 - d[k]-xs[k]*X-ys[k]*y <= -ms[k])
    @constraint(model, (X-xc[k])^2 + (y-yc[k])^2 <= d[k])
end


#Specify Objective function
@objective(model, Min, sum(ObjCoefficient[i]*w[i] for i=1:NumVar))


###CALCULATING THE MAXIMUM RADIUS OF THE CIRCLE FROM THE CENTRE TO A POINT IN THE CIRCLE

coverage1=[[1, 0, 0, 0, 1],[0, 0, 0, 1, 1],[1, 0, 0, 0, 1],[0, 0, 1, 1, 1],[0, 0, 0, 1, 1], [0, 1, 0, 0, 1],[0, 0, 1, 1, 0],[0, 0, 0, 1, 0],[1, 0, 0, 0, 0],[1, 1, 0, 0, 0],[0, 1, 0, 0, 0],[0, 0, 1, 0, 0],[0, 0, 1, 0, 0],[1, 0, 0, 0, 0],[0, 0, 0, 1, 0]]

a=[-1, -1, 1.2, 1.2, 1.2, 0.8, 2.5, 2.5, -3, -3, -1.5, 3, 0.9, -3.7, 3.7] #x cordinate of the points
b=[0.2, 0.8, -0.85, 1.2, 0.5, -1.5, -1.3, 1.4, 1.5, -1.3, -0.8, -2.7, -2.4, 0.5, 0.8] #y cordinate of the points
r=[]        # array to store max distances for the center of each circle to each point

#print("A =", coverage1)



function distance(xs, ys, a, b, coverage1) 
    r = Float64[]
    for (i,xv) in enumerate(xs)
        R = Float64[]
        for (j, jv) in enumerate(a)
            if coverage1[j][i] == 1
                push!(R, ((xs[i]-a[j])^2 + (ys[i]-b[j])^2)^0.5)
            end
         end
         if length(R) > 0
             push!(r, maximum(R))
         end
    end
    return r
end
r=distance(xs, ys, a, b, coverage1)


@show model
print(model)
optimize!(model)
@show termination_status(model)
@show primal_status(model)
@show dual_status(model)
@show objective_value(model)
for i in 1:NumVar
   println("x[$i] = ", value(x[i]))
    println("w[$i] = ", value(w[i]))
    println("r[$i] = ", value(r[i]))
end
@show value(X)
@show value(y)
@show value(R)

model = A JuMP Model
Minimization problem with:
Variables: 33
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 15 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 20 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 10 constraints
`QuadExpr`-in-`MathOptInterface.LessThan{Float64}`: 5 constraints
`Vector{VariableRef}`-in-`MathOptInterface.SecondOrderCone`: 5 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 28 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 5 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Mosek
Names registered in the model: R, X, constr, d, p, q, w, x, y, z
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 65              
  Cones                  : 10              
  Scalar variables       : 53           

373.91262010975726